<a href="https://colab.research.google.com/github/HumeraF2/AML-Assignment/blob/main/aml_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/credit_card_churn.csv")

In [ ]:
print(df.info(),'\n')
print(df.head(),'\n')
print(df.describe(),'\n')

In [ ]:
df.describe().T

In [ ]:
#Check for missing values
df.isnull().sum()

In [ ]:
#Calculate summary statistics for categorical data
for col in df.select_dtypes(include='object').columns:
    print(f"{col}:\n{df[col].value_counts()}\n")

In [ ]:
#Drop unnecessary features
df.drop(['CLIENTNUM','Marital_Status', 'Education_Level'], axis=1, inplace=True)
df.info()

In [ ]:
#Display Multiple Boxplots to identify any outliers
import matplotlib.pyplot as plt

columns = ['Customer_Age', 'Months_on_book','Total_Revolving_Bal','Credit_Limit','Avg_Open_To_Buy','Total_Trans_Amt', 'Total_Trans_Ct']
for col in columns:
  plt.boxplot(df[col],vert=False)
  plt.title(col)
  plt.ylabel(col)
  plt.rcParams["figure.figsize"] = (3,2)
  plt.show()
 

In [ ]:
#** Removing Outliers: To remove outliers, use a method like Z-score or IQR to detect and remove the extreme values. **
from scipy import stats
import scipy.stats as stats

# Calculate Z-scores and retain rows with Z-score greater than -3 or less than +3

z_scores = stats.zscore(df['Customer_Age'])
plt.boxplot(z_scores,vert=False)
df = df[(z_scores >= -3) & (z_scores <= 3)]
plt.title('Box plot of Age')
plt.ylabel('Customer_Age')
plt.rcParams["figure.figsize"] = (3,2)
plt.show()

z_scores = stats.zscore(df['Months_on_book'])
plt.boxplot(z_scores,vert=False)
df = df[(z_scores >= -3) & (z_scores <= 3)]
plt.title('Box plot of Months_on_book')
plt.ylabel('Months on Book')
plt.show()

z_scores = stats.zscore(df['Total_Trans_Ct'])
plt.boxplot(z_scores,vert=False)
df = df[(z_scores >= -3) & (z_scores <= 3)]
plt.title('Box plot of Total Transaction Ct')
plt.ylabel('Total Transaction Ct')
plt.show()

z_scores = stats.zscore(df['Credit_Limit'])
plt.boxplot(z_scores,vert=False)
df = df[(z_scores >= -3) & (z_scores <= 3)]
plt.title('Box plot of Credit Limit')
plt.ylabel('Credit Limit')
plt.show()

z_scores = stats.zscore(df['Avg_Open_To_Buy'])
plt.boxplot(z_scores,vert=False)
df = df[(z_scores >= -3) & (z_scores <= 3)]
plt.title('Box plot of Avg Open To Buy')
plt.ylabel('Avg Open To Buy')
plt.show()

z_scores = stats.zscore(df['Total_Trans_Amt'])
plt.boxplot(z_scores,vert=False)
df = df[(z_scores >= -3) & (z_scores <= 3)]
plt.title('Box plot of Total Transanction Amt')
plt.ylabel('Total Transanction Amt')
plt.show()


In [ ]:
#Display the box plot again

columns = ['Customer_Age', 'Months_on_book','Credit_Limit','Avg_Open_To_Buy','Total_Trans_Amt', 'Total_Trans_Ct']
for col in columns:
  plt.boxplot(df[col],vert=False)
  plt.title(col)
  plt.ylabel(col)
  plt.rcParams["figure.figsize"] = (3,2)
  plt.show()


In [ ]:
#Convert the categorical variables to dummy variables
df = pd.get_dummies(df, drop_first=False)
df.info()

In [ ]:
df.drop(['Attrition_Flag_Attrited Customer','Income_Category_Unknown', 'Card_Category_Silver'], axis=1, inplace=True)

In [ ]:
#Standardisation
from sklearn.preprocessing import StandardScaler
  
# Separate the dummy variables from the numerical variables
dummy_cols = df.select_dtypes(include='uint8').columns
num_cols = list(set(df.select_dtypes(include=['float64', 'int64']).columns) - set(dummy_cols))

# Standardize the numerical variables
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Combine the standardized numerical variables with the dummy variables
df_std = pd.concat([df[num_cols], df[dummy_cols]], axis=1)

print(df_std.head(5),'\n')
print(df_std.describe(),'\n')

In [ ]:
print(df.columns)

In [ ]:
#Split the data into training and test sets (70:30)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_std.drop('Attrition_Flag_Existing Customer', axis=1), df['Attrition_Flag_Existing Customer'], test_size=0.3, random_state=50)

In [ ]:
#Import/install the necessary packages
!pip install scikeras
import scikeras as sk
from keras.losses import binary_crossentropy
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense (21, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model   

# create the model
model = KerasRegressor(build_fn=create_model, verbose=0, )
print(model)

In [ ]:
# define the grid search parameters
param_grid = {'batch_size': [32, 64],
              'epochs': [50,100],
              'optimizer': ['adam', 'sgd']}


# perform grid search to find the best hyperparameters
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
 
# print the best parameters
print(f"Best Parameters: {grid_result.best_params_}")

In [ ]:
#Fit the model with the best hyperparameters and make predictions using the test set
best_model = grid.best_estimator_
best_model.fit(X_train, y_train, epochs=grid_result.best_params_['epochs'], batch_size=grid_result.best_params_['batch_size'], verbose=0, callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])

# Y predict on X test set
y_pred = best_model.predict(X_test)

In [ ]:
#Display Feature Importance

importance = best_model.predict(X_train, verbose=0)
importance = np.squeeze(importance)
feature_names = X_train.columns
feature_importance = dict(zip(feature_names, importance))
sorted_importance = sorted(feature_importance.items(), key=lambda kv: abs(kv[1]), reverse=True)

for feature, importance in sorted_importance:
    print(f"{feature}: {abs(importance)}")

importance_values = [abs(kv[1]) for kv in sorted_importance]
#print(importance_values)
feature_names = [kv[0] for kv in sorted_importance]
#print(feature_names)
plt.rcParams["figure.figsize"] = (10,5)
plt.barh(range(len(importance_values)), importance_values, align='center')
plt.yticks(range(len(feature_names)), feature_names)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance")
plt.show()

In [ ]:
#predict the churn values
print(y_pred)
# unscaling the y_pred values 
y_pred_lis = []
for i in y_pred:
    if i>0.5:
        y_pred_lis.append(1)
    else:
        y_pred_lis.append(0)
print(y_pred_lis)

In [ ]:
#make dataframe for comparing the orignal and predict values
data = {'orignal_churn':y_test, 'predicted_churn':y_pred_lis}
df_check = pd.DataFrame(data)
df_check.head(10)

In [ ]:
# checking for performance metrices, importing classification_report and confusion metrics
import seaborn as sb
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

#print classification_report
print(classification_report(y_test, y_pred_lis))

# ploting the confusion metrix plot
conf_mat = tf.math.confusion_matrix(labels=y_test,predictions=y_pred_lis)
plt.figure(figsize = (10,5))
sb.heatmap(conf_mat, annot=True,fmt='d',)
plt.xlabel('Predicted_number')
plt.ylabel('True_number')

In [ ]:
#Roc and auc curves
from sklearn import metrics
auc = metrics.roc_auc_score(y_test, y_pred)
false_positive_rate, true_positive_rate, thresolds = metrics.roc_curve(y_test, y_pred)

plt.figure(figsize=(6, 4), dpi=100)
plt.axis('scaled')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.title("AUC & ROC Curve")
plt.plot(false_positive_rate, true_positive_rate, 'g')
plt.fill_between(false_positive_rate, true_positive_rate, facecolor='lightgreen', alpha=0.7)
plt.text(0.95, 0.05, 'AUC = %0.4f' % auc, ha='right', fontsize=12, weight='bold', color='blue')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()